# Day 12: Cross-Silo Federated Learning for Banks

**Real-World FL Deployment Across Financial Institutions**

## Overview
- **Scenario**: Multiple banks collaborating on fraud detection
- **Challenge**: Regulatory constraints, data privacy, model ownership
- **Solution**: Cross-silo FL with Flower framework

## What You'll Learn
1. **Cross-Silo Architecture**: FL across institutions
2. **Regulatory Compliance**: GDPR, data governance
3. **Deployment Patterns**: Server setup, client registration
4. **Practical Considerations**: Network, security, monitoring

---

## 1. Cross-Silo FL Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                   Central FL Server                         │
│  - Coordinates training rounds                              │
│  - Aggregates bank updates                                  │
│  - Manages model versions                                   │
└────────────┬────────────────────────────────────────────────┘
             │
    ┌────────┼────────┐
    │        │        │
┌───▼────┐ ┌─▼──────┐ ┌▼─────────┐
│ Bank A │ │ Bank B │ │ Bank C   │
│        │ │        │ │          │
│ - Fraud│ │ - Fraud│ │ - Fraud  │
│   data │ │   data │ │   data   │
│ - FL   │ │ - FL   │ │ - FL     │
│   client││   client│ │   client│
└────────┘ └────────┘ └──────────┘
```

**Key Differences from Cross-Device:**
- **Reliable connectivity**: Banks have stable internet
- **More data per client**: Each bank has millions of transactions
- **Regulatory oversight**: Compliance, audit trails required
- **Trust model**: Semi-trusted environment (contractual agreements)

## 2. Bank-Specific Data Partitioning

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Cross-Silo FL for Banks")
print("\nScenario: 5 regional banks collaborating on fraud detection")
print("\nBanks:")
banks = [
    "Bank A (North America)",
    "Bank B (Europe)",
    "Bank C (Asia Pacific)",
    "Bank D (Latin America)",
    "Bank E (Middle East)",
]
for i, bank in enumerate(banks):
    print(f"  {i}: {bank}")

## 3. Regional Fraud Pattern Variation

In [ ]:
# Simulate regional fraud patterns
np.random.seed(42)

# Each bank has different fraud characteristics
bank_fraud_patterns = {
    'Bank A': {'base_rate': 0.02, 'peak_hour': 14, 'avg_amount': 450},
    'Bank B': {'base_rate': 0.03, 'peak_hour': 10, 'avg_amount': 520},
    'Bank C': {'base_rate': 0.04, 'peak_hour': 22, 'avg_amount': 380},
    'Bank D': {'base_rate': 0.025, 'peak_hour': 18, 'avg_amount': 490},
    'Bank E': {'base_rate': 0.035, 'peak_hour': 16, 'avg_amount': 510},
}

# Generate data for each bank
banks_data = {}
for bank, pattern in bank_fraud_patterns.items():
    n_samples = 50000
    
    # Features
    amounts = np.random.lognormal(6, 1, n_samples)
    hours = np.random.randint(0, 24, n_samples)
    merchants = np.random.randint(0, 100, n_samples)
    
    # Fraud labels (regional pattern)
    fraud_prob = (
        pattern['base_rate'] * 1.5 +  # Base rate
        0.01 * np.exp(-((hours - pattern['peak_hour'])**2) / 20) +  # Time-based
        0.02 * (amounts > np.percentile(amounts, 95)).astype(float)  # Amount-based
    )
    labels = (np.random.random(n_samples) < fraud_prob).astype(int)
    
    banks_data[bank] = {
        'X': np.column_stack([amounts, hours, merchants]),
        'y': labels,
        'fraud_rate': labels.mean()
    }

# Display statistics
print("\n" + "="*60)
print("BANK FRAUD STATISTICS")
print("="*60)
for bank, data in banks_data.items():
    pattern = bank_fraud_patterns[bank]
    print(f"\n{bank}:")
    print(f"  Samples: {len(data['y']):,}")
    print(f"  Fraud rate: {data['fraud_rate']*100:.2f}%")
    print(f"  Peak fraud hour: {pattern['peak_hour']}:00")
    print(f"  Avg fraud amount: ${pattern['avg_amount']:.0f}")

## 4. Cross-Silo FL Training

In [ ]:
# Conceptual cross-silo FL training loop
print("\n" + "="*60)
print("CROSS-SILO FL TRAINING WORKFLOW")
print("="*60)

workflow = '''

1. INITIALIZATION
   - Server initializes global fraud detection model
   - Registers participating banks (5 clients)
   - Establishes secure TLS connections

2. FEDERATED ROUND (repeated 50 times)
   ┌─────────────────────────────────────────────────┐
   │ Server                                           │
   │ • Selects all banks (cross-silo: 100% participation)│
   │ • Sends global model weights (v45.2 MB)          │
   └─────────────────────────────────────────────────┘
           │              │              │
           ▼              ▼              ▼
       ┌───────┐    ┌───────┐    ┌───────┐
       │Bank A │    │Bank B │    │Bank C │ ...
       └───────┘    └───────┘    └───────┘
           │            │            │
           ▼            ▼            ▼
       • Train    • Train     • Train
         5 epochs    5 epochs    5 epochs
       • Compute  • Compute   • Compute
         update      update     update
           │            │            │
           ▼            ▼            ▼
   ┌─────────────────────────────────────────────────┐
   │ Server: Aggregate updates (FedAvg)              │
   │ • Weight by bank data size                       │
   │ • Update global model                           │
   │ • Broadcast new weights                          │
   └─────────────────────────────────────────────────┘

3. EVALUATION
   - Each bank evaluates global model on local test set
   - Server collects metrics (accuracy, precision, recall)
   - Model version recorded for audit trail

4. DEPLOYMENT
   - Global model deployed to each bank's fraud detection system
   - Banks retain ownership of their data
   - Regulatory compliance satisfied

'''

print(workflow)

print("KEY CHARACTERISTICS:")
print("  • All banks participate every round (reliable connectivity)")
print("  • Larger local epochs (5-10) due to more data per bank")
print("  • Strong security requirements (TLS, authentication)")
print("  • Regulatory compliance (audit logs, model governance)")

## 5. Regulatory Compliance Requirements

In [ ]:
# Compliance considerations
compliance_df = pd.DataFrame({
    'Regulation': [
        'GDPR (EU)',
        'CCPA (California)',
        'SOX (Banking)',
        'PCI DSS',
        'Basel III'
    ],
    'Requirement': [
        'Data cannot leave EU, right to explanation',
        'Opt-out of data selling, data deletion rights',
        'Audit trails, model validation',
        'Secure data transmission, access control',
        'Model risk management, stress testing'
    ],
    'FL Compliance': [
        '✅ Data stays local',
        '✅ No data sharing',
        '✅ Model versioning, logs',
        '✅ Encrypted comms',
        '✅ Model validation possible'
    ]
})

print("\n" + "="*70)
print("REGULATORY COMPLIANCE IN CROSS-SILO FL")
print("="*70)
print(compliance_df.to_string(index=False))

## 6. Deployment Architecture

In [ ]:
deployment_architecture = '''

                        ┌──────────────────┐
                        │  Load Balancer   │
                        └────────┬─────────┘
                                 │
                        ┌────────▼─────────┐
                        │  FL Server Node   │
                        │  - Flower Server  │
                        │  - Model Store    │
                        │  - Metrics DB     │
                        │  - Audit Log      │
                        └────────┬─────────┘
                                 │
                ┌────────────────┼────────────────┐
                │                │                │
         ┌──────▼──────┐  ┌─────▼─────┐  ┌─────▼─────┐
         │  Database   │  │  Object   │  │  Logging  │
         │  (Postgres) │  │  Store    │  │  (ELK)    │
         └─────────────┘  │  (S3)     │  └───────────┘
                          └───────────┘

Network Security:
  • TLS 1.3 for all communication
  • Mutual authentication (mTLS)
  • VPN between bank data centers
  • Firewall rules, IP whitelisting

Monitoring:
  • Round-by-round accuracy
  • Per-bank contribution tracking
  • Anomaly detection on updates
  • Alert system for failures

Governance:
  • Model approval workflow
  • Rollback capability
  • A/B testing framework
  • Explanations for regulators

'''

print(deployment_architecture)

## 7. Summary

### Cross-Silo FL Key Points:

**Architecture:**
- 5-100 organizations (banks, hospitals)
- Reliable, high-speed connectivity
- Semi-trusted environment (contracts)
- Strong security (TLS, authentication)

**Training Characteristics:**
- All clients participate (100%)
- More local epochs (5-10)
- Larger models (deeper networks)
- Longer rounds (minutes to hours)

**Compliance:**
- ✅ Data never leaves premises
- ✅ Audit trails for all operations
- ✅ Model governance and versioning
- ✅ Regulatory-friendly

**Use Cases:**
- Financial fraud detection (banks)
- Medical imaging (hospitals)
- Supply chain (companies)
- Drug discovery (pharma)

### Next Steps:
→ **Day 13**: Vertical Federated Learning (feature partitioning)
→ **Day 19**: FoolsGold Defense (Sybil-resistant aggregation)

---

**📁 Project Location**: `02_federated_learning_foundations/cross_silo_bank_fl/`